<h2> Preprocessing data

<h4> В данном ноутбуке предобрабатываются текста в надлежащий вид, а именно:
<h5> 1) Заменяются кодировки на символы, соответствующие коду
<h5> 1) Заменяются ссылки и почты на токен email и url
<h5> 2) Удаляются ссылки на источники ([1], [2], [3] и т.д.)
<h5> 3) Понижается регистр
<h5> 4) Удаляются знаки пунктуации

In [1]:
import pandas as pd
import html
import string

<h5>  Перевод json-файла с обучающей выборкой в Pandas DataFrame

In [2]:
data_train = pd.read_json('ranking_train.jsonl', lines=True)
new_rows = []
for index, row in data_train.iterrows():
    new_row = {'text': row['text']}
    for i in range(len(row['comments'])):
        new_row['comments_text_' + str(i)] = row['comments'][i]['text']
    new_rows.append(new_row)

pd.DataFrame(new_rows).to_csv('ranking_train.tsv', sep="\t", index=False)

In [3]:
train_data_tsv = pd.read_csv('ranking_train.tsv', sep="\t")
train_data_tsv[:5]

,text,comments_text_0,comments_text_1,comments_text_2,comments_text_3,comments_text_4
0,How many summer Y Combinator fundees decided n...,Going back to school is not identical with giv...,There will invariably be those who don't see t...,For me school is a way to be connected to what...,I guess it really depends on how hungry you ar...,I know pollground decided to go back to school...
1,CBS acquires last.fm for $280m,It will be curious to see where this heads in ...,Does this mean that there's now a big-name com...,Also on BBC News: http://news.bbc.co.uk/1/low...,I don't understand what they do that is worth ...,sold out too cheaply. given their leadership p...
2,How Costco Became the Anti-Wal-Mart,I really hate it when people falsely claim tha...,"I love Costco. For me, the best quote of the ...","""But Mr. Sinegal warned that if Costco increas...",While Costco does treat its employees a little...,"I'd like to know more about their ""don't adver..."
3,"Fortune Favors Big Turds | Screw The Money, Th...","His real point is that something can be ""simpl...","So I'm thinking, ""Well, I've never heard of th...",I've never seen a single blog post wander in s...,It was a RANT on a personal blog. It's there t...,"In the eternal words of the Geico Caveman..""Wh..."
4,StartupWeekend: 70 Founders Create One Company...,Looks like someone hasn't read The Mythical Ma...,Any chances for recreating this in a telecommu...,"You know, if they started with 70 and eliminat...",And what did you do this weekend?I'd love to h...,Very interesting. I would join an event like t...


<h5> Предобработка текстовых данных с использованием замен и удалений ненужных символов

In [4]:
for column in train_data_tsv.columns:
    #replacing codes with symbols
    train_data_tsv[column] = train_data_tsv[column].apply(html.unescape)

    #replacing urls with the <url> token
    train_data_tsv[column] = train_data_tsv[column].str.replace(
    r"(http|ftp|https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-])", 
    ' <url> ',
    regex=True)

    #replacing emais with the <email> token
    train_data_tsv[column] = train_data_tsv[column].str.replace(
        r"(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*)@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9]))\.){3}(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9])|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])",
        ' <email> ',
        regex=True)
    
    #removing links to sources
    train_data_tsv[column] = train_data_tsv[column].str.replace(r'\[\d\]', '', regex=True)
    
    # lowering the register
    # train_data_tsv[column] = train_data_tsv[column].apply(lambda x: x.lower())

    # removing punctuation marks
    # train_data_tsv[column] = train_data_tsv[column].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation + "\n")))

In [5]:
train_data_tsv.to_csv('ranking_train.tsv', sep="\t", index=False)
train_data_tsv = pd.read_csv('ranking_train.tsv', sep="\t")
train_data_tsv[:5]

,text,comments_text_0,comments_text_1,comments_text_2,comments_text_3,comments_text_4
0,How many summer Y Combinator fundees decided n...,Going back to school is not identical with giv...,There will invariably be those who don't see t...,For me school is a way to be connected to what...,I guess it really depends on how hungry you ar...,I know pollground decided to go back to school...
1,CBS acquires last.fm for $280m,It will be curious to see where this heads in ...,Does this mean that there's now a big-name com...,Also on BBC News: <url> .Nice to see a Lond...,I don't understand what they do that is worth ...,sold out too cheaply. given their leadership p...
2,How Costco Became the Anti-Wal-Mart,I really hate it when people falsely claim tha...,"I love Costco. For me, the best quote of the ...","""But Mr. Sinegal warned that if Costco increas...",While Costco does treat its employees a little...,"I'd like to know more about their ""don't adver..."
3,"Fortune Favors Big Turds | Screw The Money, Th...","His real point is that something can be ""simpl...","So I'm thinking, ""Well, I've never heard of th...",I've never seen a single blog post wander in s...,It was a RANT on a personal blog. It's there t...,"In the eternal words of the Geico Caveman..""Wh..."
4,StartupWeekend: 70 Founders Create One Company...,Looks like someone hasn't read The Mythical Ma...,Any chances for recreating this in a telecommu...,"You know, if they started with 70 and eliminat...",And what did you do this weekend?I'd love to h...,Very interesting. I would join an event like t...


<h5> Переформирование датасета для удобства выгрузки в методe c TF-IDF 

In [6]:
data_tf_idf_list = []

for i in range(len(train_data_tsv)): 
    for column in train_data_tsv.columns:
        data_tf_idf_list.append(train_data_tsv[column][i])

pd.DataFrame(data_tf_idf_list).to_csv('ranking_train_tf_idf.tsv', sep="\t", index=False)

In [7]:
train_data_tsv_tf_idf = pd.read_csv('ranking_train_tf_idf.tsv', sep="\t")
train_data_tsv_tf_idf[:10]

,0
0,How many summer Y Combinator fundees decided n...
1,Going back to school is not identical with giv...
2,There will invariably be those who don't see t...
3,For me school is a way to be connected to what...
4,I guess it really depends on how hungry you ar...
5,I know pollground decided to go back to school...
6,CBS acquires last.fm for $280m
7,It will be curious to see where this heads in ...
8,Does this mean that there's now a big-name com...
9,Also on BBC News: <url> .Nice to see a Lond...


<h5> Аналогичная процедура создания Pandas DataFrame для тестовой выборки

In [8]:
data_test = pd.read_json('ranking_test.jsonl', lines=True)
new_rows = []
for index, row in data_test.iterrows():
    new_row = {'text': row['text']}
    for i in range(len(row['comments'])):
        new_row['comments_text_' + str(i)] = row['comments'][i]['text']
    new_rows.append(new_row)

pd.DataFrame(new_rows).to_csv('ranking_test.tsv', sep="\t", index=False)

In [9]:
test_data_tsv = pd.read_csv('ranking_test.tsv', sep="\t")
test_data_tsv[:5]

,text,comments_text_0,comments_text_1,comments_text_2,comments_text_3,comments_text_4
0,"iOS 8.0.1 released, broken on iPhone 6 models,...",I&#x27;m still waiting for them to stabilize w...,"For those who upgraded, no need to do a restor...",Upgraded shortly after it was released and suf...,I think they were under a lot of pressure on t...,Fix for those who already updated: http:&#x2F...
1,Ask HN: How do US HNers get their health insur...,Get it from your employer. It&#x27;s better an...,As someone who moved from the UK to US under a...,"I went to http:&#x2F;&#x2F;healthcare.gov, fil...","Full time employed, health insurance through m...",Your options depend on where you live.I signed...
2,San Diego Researcher Crowdfunding Patent-Free ...,What I don&#x27;t understand is why drugs deve...,To everyone sure this will fail because of the...,People don&#x27;t seem to realize how expensiv...,"Many years ago, I worked in a lab where I synt...",I think the negative comments on here have got...
3,Rethinking the origins of the universe,I&#x27;m not a physicist. I imagine that if I...,I&#x27;m not at all acquainted with astrophysi...,This paper claims that black holes can&#x27;t ...,Interesting; if the paper finds general accept...,Ethan Siegel just posted a detailed response t...
4,SlackTextViewController: A new growing text in...,As someone that doesn&#x27;t do iOS developmen...,Apple developed Swift to make developer&#x27;s...,"So I&#x27;m making iOS apps sometimes, but hav...",&gt; Inverted mode for displaying cells upside...,cool. I was just shopping for a new growing t...


In [10]:
for column in test_data_tsv.columns:
    #replacing codes with symbols
    test_data_tsv[column] = test_data_tsv[column].apply(html.unescape)

    #replacing urls with the <url> token
    test_data_tsv[column] = test_data_tsv[column].str.replace(
    r"(http|ftp|https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-])", 
    ' <url> ',
    regex=True)

    #replacing emais with the <email> token
    test_data_tsv[column] = test_data_tsv[column].str.replace(
        r"(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*)@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9]))\.){3}(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9])|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])",
        ' <email> ',
        regex=True)
    
    #removing links to sources
    test_data_tsv[column] = test_data_tsv[column].str.replace(r'\[\d\]', '', regex=True)
    
    # lowering the register
    # test_data_tsv[column] = test_data_tsv[column].apply(lambda x: x.lower())

    # removing punctuation marks
    # test_data_tsv[column] = test_data_tsv[column].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation + "\n")))


In [11]:
test_data_tsv.to_csv('ranking_test.tsv', sep="\t", index=False)
test_data_tsv = pd.read_csv('ranking_test.tsv', sep="\t")
test_data_tsv[:5]

,text,comments_text_0,comments_text_1,comments_text_2,comments_text_3,comments_text_4
0,"iOS 8.0.1 released, broken on iPhone 6 models,...",I'm still waiting for them to stabilize wifi o...,"For those who upgraded, no need to do a restor...",Upgraded shortly after it was released and suf...,I think they were under a lot of pressure on t...,Fix for those who already updated: <url> ...
1,Ask HN: How do US HNers get their health insur...,Get it from your employer. It's better and che...,As someone who moved from the UK to US under a...,"I went to <url> , filled out some online form...","Full time employed, health insurance through m...",Your options depend on where you live.I signed...
2,San Diego Researcher Crowdfunding Patent-Free ...,What I don't understand is why drugs developed...,To everyone sure this will fail because of the...,People don't seem to realize how expensive it ...,"Many years ago, I worked in a lab where I synt...",I think the negative comments on here have got...
3,Rethinking the origins of the universe,I'm not a physicist. I imagine that if I were...,I'm not at all acquainted with astrophysics bu...,This paper claims that black holes can't come ...,Interesting; if the paper finds general accept...,Ethan Siegel just posted a detailed response t...
4,SlackTextViewController: A new growing text in...,As someone that doesn't do iOS development it ...,Apple developed Swift to make developer's more...,"So I'm making iOS apps sometimes, but have not...",> Inverted mode for displaying cells upside-do...,cool. I was just shopping for a new growing t...
